In [1]:
from llama_index.core import VectorStoreIndex
from llama_index.core.agent import AgentRunner, FunctionCallingAgentWorker
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding, OpenAIEmbeddingModelType
from llama_index.core.tools import QueryEngineTool
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool, QueryEngineTool, ToolMetadata
from llama_index.core import Settings

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
llm = OpenAI(model="gpt-4o-mini")
embed_model = OpenAIEmbedding(model=OpenAIEmbeddingModelType.TEXT_EMBED_3_SMALL)
Settings.embed_model = embed_model
Settings.llm = llm

# Improvements
- Build a dynamic chunking algorithm based on the given document considering number of pages, words, images in doc, etc.

In [4]:
documents = SimpleDirectoryReader(input_files=["handbook.pdf"]).load_data()

In [5]:
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=64)
nodes = splitter.get_nodes_from_documents(documents)

In [6]:
len(nodes)

86

In [7]:
index = VectorStoreIndex(
    nodes=nodes,
)

In [8]:
qe = index.as_query_engine(use_async=True)

### Sample Tests

In [62]:
response = qe.query("What is the name of the company?")

In [63]:
str(response)

'Zania, Inc.'

In [64]:
for node in response.source_nodes:
    print(node)

Node ID: 3742cc7d-3b65-4bed-bbdd-1108e8cb9c3f
Text: Software programs purchased and provided by Zania, Inc. are to
be used only for creating, researching, and processing materials for
Company use. By using Company hardware, software, and networking
systems you assume personal responsibility for their use and agree to
comply with this policy and other applicable Company policies, as well
as city, ...
Score:  0.347

Node ID: 1b59923c-e083-463f-9642-f48b61d03bdb
Text: financial or sales records/reports, marketing or business
strategies/plans, product development, customer lists, patents,
trademarks, etc.) related to the Company. Do not create a link from
your personal blog, website, or other social networking site to a
Company website that identifies you as speaking on behalf of the
Company. Never represent y...
Score:  0.342



In [65]:
from pprint import pprint
import json

In [66]:
pprint(response.source_nodes[0].to_dict())

{'class_name': 'NodeWithScore',
 'node': {'class_name': 'TextNode',
          'embedding': None,
          'end_char_idx': 2451,
          'excluded_embed_metadata_keys': ['file_name',
                                           'file_type',
                                           'file_size',
                                           'creation_date',
                                           'last_modified_date',
                                           'last_accessed_date'],
          'excluded_llm_metadata_keys': ['file_name',
                                         'file_type',
                                         'file_size',
                                         'creation_date',
                                         'last_modified_date',
                                         'last_accessed_date'],
          'id_': '3742cc7d-3b65-4bed-bbdd-1108e8cb9c3f',
          'metadata': {'creation_date': '2024-11-07',
                       'file_name': 'handbook.pdf',
  

In [67]:
response2 = qe.query("Who is the CEO of the company?")

In [68]:
str(response2)

'The CEO of the company is Shruti Gupta.'

In [69]:
for node in response2.source_nodes:
    print(node)

Node ID: 071a4c9f-8f5e-4ab9-bfa4-f66417a458e9
Text: Closing Statement Thank you for reading our handbook. We hope it
has provided you with an understanding of our mission, history, and
structure as well as our current policies and guidelines. We look
forward to working with you to create a successful Company and a safe,
productive, and pleasant workplace. Shruti Gupta, CEO Zania, Inc. 45
Score:  0.369

Node ID: 20a40c29-c644-4687-81f7-a11dbcbf1d97
Text: is on an "at-will" basis. This means your employment may be
terminated at any time, with or without notice and with or without
cause. Likewise, we respect your right to leave the Company at any
time, with or without notice and with or without cause.  Nothing in
this handbook or any other Company document should be understood as
creating a contra...
Score:  0.348



In [70]:
response3 = qe.query("What is their vacation policy?")

In [71]:
str(response3)

"The vacation policy includes the following key points:\n\n1. Vacation is prorated during the first year of employment based on the hire date.\n2. Eligible employees accrue a specified amount of vacation for every period worked, up to a maximum accrual limit. Once this limit is reached, no additional vacation will accrue until some of the accrued vacation is used.\n3. Employees can begin using vacation immediately upon hire or after a specified introductory period, depending on the company's policy.\n4. Requests for vacation should be made in advance, with a minimum notice period required.\n5. Vacation must be taken in specified increments.\n6. Unused vacation may be required to be used during certain leaves of absence, as permitted by law.\n7. There are options for carrying over unused vacation to the following year or for receiving payment for unused time at specified intervals.\n\nThe specifics regarding the amount of vacation, accrual periods, and other details would depend on the 

In [72]:
for node in response3.source_nodes:
    print(node)

Node ID: 2a148360-810f-4f73-945c-b3ad5f90e1be
Text: Vacation granted during your first year of employment will be
prorated based on your hire date. [[ OR ]] [[ Option 2 :]] All
eligible employees will accrue [[# hours/days/weeks]] of vacation for
every [[period of time]] worked, up to a maximum accrual of [[#
hours/days/weeks]].  Once you reach the maximum accrual amount, you
will not accrue any ...
Score:  0.541

Node ID: e2e0daf5-355e-4136-a102-56d82f8cc26f
Text: Deposits Into Your Leave Account Vacation is calculated
according to [[your work anniversary year/the calendar year/the fiscal
year, which begins on [date] and ends on [date]]].  [[ EMPLOYERS MUST
CHOOSE ONE :]] [[ Option 1 :]] The amount of vacation received each
year is based on your length of service and [[is granted in a lump sum
at the begi...
Score:  0.528



In [73]:
response4 = qe.query("What is the termination policy?")

In [74]:
str(response4)

'The termination policy includes several key points. Involvement in criminal activity, whether on or off Company property, may lead to disciplinary action, including suspension or termination. Employees are expected to be present and ready to work as scheduled; failure to do so may result in disciplinary action, including termination for attendance policy violations or job abandonment. Additionally, violations of company policies or procedures can lead to disciplinary actions such as demotion, transfer, leave without pay, or termination. The company encourages progressive discipline but is not obligated to follow this approach and may terminate employees for conduct violations or inadequate work performance.'

In [75]:
response5 = qe.query("Who are the competitors to this organization?")

In [76]:
str(response5)

'The provided information does not include any details about competitors to the organization.'

In [77]:
for node in response5.source_nodes:
    print(node)

Node ID: 7aabac05-ef8b-4aee-863f-e9e01a763e43
Text: Retaliation and Your Rights Retaliation or any other negative
action is prohibited against anyone who, based on a reasonable belief,
reports a possible deviation from this policy or cooperates in an
investigation. Those who retaliate against others for reporting a
possible deviation from this policy or for cooperating in an
investigation will be...
Score:  0.190

Node ID: c4a139e8-47f9-4fdf-b35e-730d6e40b1c7
Text: Company may consider your continued employment as long as
concerns regarding safety, health, production, communication, or other
work-related matters are adequately addressed. The Company may also
require you to obtain a medical clearance and agree to random testing
and a "one-strike" rule as a condition of continued employment.]]
Violations Vio...
Score:  0.189



### Continue

In [9]:
doc_qe_tool = QueryEngineTool(
    query_engine=qe,
    metadata=ToolMetadata(
        name="Document Q&A",
        description="Provides information about the provided document and answers questions specific to the provided document."
    )
)

In [10]:
import nest_asyncio

nest_asyncio.apply()

In [41]:
from llama_index.core import load_index_from_storage

In [12]:
from llama_index.core.agent import StructuredPlannerAgent

In [13]:
from typing import Optional, List

In [14]:
def vector_query(
        query: str
    ) -> str:
        """Use to answer questions over a given document.
    
        Useful if you have specific questions over the document.
    
        Args:
            query (str): the string query to be embedded.        
        """
           
        query_engine = index.as_query_engine(
            similarity_top_k=2,
        )
        response = query_engine.query(query)
        return response

In [154]:
from typing import Any
from llama_index.core.tools import ToolOutput

class ControlledFunctionTool(FunctionTool):

    def call(self, *args: Any, **kwargs: Any) -> ToolOutput:
            """Call."""
            response.sources[0].raw_output.source_nodes
            tool_output = self._fn(*args, **kwargs)
            content = ""
            if tool_output.source_nodes:
                score = max([node.score for node in tool_output.source_nodes])
                if score < 0.2: # The threshould for relavent info in document found or not
                    tool_output.response = "Data Not Available"
            
            return ToolOutput(
                content=str(tool_output),
                tool_name=self.metadata.name,
                raw_input={"args": args, "kwargs": kwargs},
                raw_output=tool_output,
            )

In [155]:
doc_vector_tool = ControlledFunctionTool.from_defaults(
    fn=vector_query,
    name="document_vector_tool",
    description="This is the tool to helpful for answering a single question at a time on the provided document."
)

In [40]:
index.storage_context.persist("./data/handbook")

In [43]:
from llama_index.core import StorageContext

In [44]:
index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir="./data/handbook")
    )

In [96]:
from llama_index.core.llms import ChatMessage

In [156]:
prefix_messages = [
    ChatMessage(
        role="system",
        content=(
            f"You are now an agent answering to the questions one by one, only pertaining to the provided indexed document. "
            "Only answer the questions that the user has explicitly provided. "
            "The final response should be a json with indivudual questions as keys and the corresponding function output as value. "
            "Do not make up any details."
        ),
    )
]

In [170]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [doc_vector_tool],
    verbose=True,
    prefix_messages=prefix_messages
)

In [171]:
agent = AgentRunner(agent_worker)

In [172]:
resp = agent.query("What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n Who is elon musk?")

Added user message to memory: What is the name of the company? 
 Who is the CEO of the company? 
 What is their vacation policy? 
 Who is elon musk?
=== Calling Function ===
Calling function: document_vector_tool with args: {"query": "name of the company"}
=== Function Output ===
Zania, Inc.
=== Calling Function ===
Calling function: document_vector_tool with args: {"query": "CEO of the company"}
=== Function Output ===
Shruti Gupta is the CEO of Zania, Inc.
=== Calling Function ===
Calling function: document_vector_tool with args: {"query": "vacation policy"}
=== Function Output ===
The vacation policy includes the following key points:

1. Vacation granted during the first year of employment is prorated based on the hire date.
2. Eligible employees accrue vacation based on hours, days, or weeks worked, up to a maximum limit. Once this limit is reached, no additional vacation will accrue until some of the accrued vacation is used.
3. Employees can begin using vacation immediately upon

In [159]:
task = agent.create_task("What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n Who is elon musk?")

In [160]:
task.task_id

'ae04cdc8-eb3b-4a03-b1fb-23193edb8ba9'

In [161]:
len(agent.get_upcoming_steps(task.task_id))

1

In [162]:
step_output = agent.run_step(task.task_id)

Added user message to memory: What is the name of the company? 
 Who is the CEO of the company? 
 What is their vacation policy? 
 Who is elon musk?
=== Calling Function ===
Calling function: document_vector_tool with args: {"query": "name of the company"}
=== Function Output ===
Zania, Inc.
=== Calling Function ===
Calling function: document_vector_tool with args: {"query": "CEO of the company"}
=== Function Output ===
Shruti Gupta is the CEO of Zania, Inc.
=== Calling Function ===
Calling function: document_vector_tool with args: {"query": "vacation policy"}
=== Function Output ===
The vacation policy includes the following key points:

1. Vacation granted during the first year of employment is prorated based on the hire date.
2. Eligible employees accrue vacation at a specified rate for every period worked, up to a maximum amount. Once this maximum is reached, no additional vacation will accrue until some of the accrued vacation is used.
3. Employees can begin using vacation immedia

In [163]:
step_output.output.sources

[ToolOutput(content='Zania, Inc.', tool_name='document_vector_tool', raw_input={'args': ('name of the company',), 'kwargs': {}}, raw_output=Response(response='Zania, Inc.', source_nodes=[NodeWithScore(node=TextNode(id_='a0a0a99b-db00-4043-9660-f0f5c5b00cfe', embedding=None, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ff06e053-7b6e-4e15-ad06-8a98f10415d6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'appli

In [164]:
step_output.output.source_nodes

[NodeWithScore(node=TextNode(id_='a0a0a99b-db00-4043-9660-f0f5c5b00cfe', embedding=None, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ff06e053-7b6e-4e15-ad06-8a98f10415d6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, hash='358037d7996c62e29453cda30b7ee338b1fff519507f0bf0e2b8864741d0c46d'), <NodeRelat

In [165]:
step_output2 = agent.run_step(task.task_id)

=== LLM Response ===
{
  "name of the company": "Zania, Inc.",
  "CEO of the company": "Shruti Gupta is the CEO of Zania, Inc.",
  "vacation policy": "The vacation policy includes the following key points:\n\n1. Vacation granted during the first year of employment is prorated based on the hire date.\n2. Eligible employees accrue vacation at a specified rate for every period worked, up to a maximum amount. Once this maximum is reached, no additional vacation will accrue until some of the accrued vacation is used.\n3. Employees can begin using vacation immediately upon hire or after a specified period, and requests for vacation should be made in advance.\n4. Vacation must be taken in increments of at least a specified number of hours or days.\n5. Unused vacation may be carried over to the following year, or the company may offer payment for unused vacation at a designated time.\n6. Upon separation from employment, employees may forfeit any earned but unused vacation unless state law prov

In [166]:
agent.state.get_task(task.task_id).__dict__

{'task_id': 'ae04cdc8-eb3b-4a03-b1fb-23193edb8ba9',
 'input': 'What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n Who is elon musk?',
 'memory': ChatMemoryBuffer(chat_store=SimpleChatStore(store={}), chat_store_key='chat_history', token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all')),
 'callback_manager': <llama_index.core.callbacks.base.CallbackManager at 0x1f771b433a0>,
 'extra_state': {'sources': [ToolOutput(content='Zania, Inc.', tool_name='document_vector_tool', raw_input={'args': ('name of the company',), 'kwargs': {}}, raw_output=Response(response='Zania, Inc.', source_nodes=[NodeWithScore(node=TextNode(id_='a0a0a99b-db00-4043-9660-f0f5c5b00cfe', embedding=None, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_dat

In [167]:
len(agent.get_upcoming_steps(task.task_id))

0

In [168]:
response = agent.finalize_response(task.task_id)

In [169]:
import json
json.loads(response.response)

{'name of the company': 'Zania, Inc.',
 'CEO of the company': 'Shruti Gupta is the CEO of Zania, Inc.',
 'vacation policy': 'The vacation policy includes the following key points:\n\n1. Vacation granted during the first year of employment is prorated based on the hire date.\n2. Eligible employees accrue vacation at a specified rate for every period worked, up to a maximum amount. Once this maximum is reached, no additional vacation will accrue until some of the accrued vacation is used.\n3. Employees can begin using vacation immediately upon hire or after a specified period, and requests for vacation should be made in advance.\n4. Vacation must be taken in increments of at least a specified number of hours or days.\n5. Unused vacation may be carried over to the following year, or the company may offer payment for unused vacation at a designated time.\n6. Upon separation from employment, employees may forfeit any earned but unused vacation unless state law provides otherwise.',
 'Who is

[NodeWithScore(node=TextNode(id_='a0a0a99b-db00-4043-9660-f0f5c5b00cfe', embedding=None, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ff06e053-7b6e-4e15-ad06-8a98f10415d6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, hash='358037d7996c62e29453cda30b7ee338b1fff519507f0bf0e2b8864741d0c46d'), <NodeRelat

In [142]:
response.sources[0].raw_output.source_nodes

[NodeWithScore(node=TextNode(id_='a0a0a99b-db00-4043-9660-f0f5c5b00cfe', embedding=None, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ff06e053-7b6e-4e15-ad06-8a98f10415d6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '15', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, hash='358037d7996c62e29453cda30b7ee338b1fff519507f0bf0e2b8864741d0c46d'), <NodeRelat

In [143]:
sum([node.score for node in response.sources[1].raw_output.source_nodes])

0.7372114382614361

In [144]:
sum([node.score for node in response.sources[0].raw_output.source_nodes])

0.7478830086564805

In [145]:
sum([node.score for node in response.sources[2].raw_output.source_nodes])

1.1642456615837546

In [146]:
sum([node.score for node in response.sources[3].raw_output.source_nodes])

0.34232271358908783

In [153]:
response.sources[3].raw_output

Response(response='The provided information does not contain any details about Elon Musk.', source_nodes=[NodeWithScore(node=TextNode(id_='bf73cf73-5e9c-42f1-8762-1e5f6b84ddd2', embedding=None, metadata={'page_label': '18', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='38beb029-a70d-420b-a68f-f7d43ffe524d', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '18', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_d

In [108]:
agent.query("Who is the CEO of the company?")

Added user message to memory: Who is the CEO of the company?
=== Calling Function ===
Calling function: vector_tool with args: {"query": "CEO of the company"}
=== Function Output ===
Shruti Gupta is the CEO of Zania, Inc.
=== LLM Response ===
The CEO of Zania, Inc. is Shruti Gupta.


Response(response='The CEO of Zania, Inc. is Shruti Gupta.', source_nodes=[NodeWithScore(node=TextNode(id_='071a4c9f-8f5e-4ab9-bfa4-f66417a458e9', embedding=None, metadata={'page_label': '45', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2a88b4c2-ec83-47f9-b99f-f09672f84b75', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '45', 'file_name': 'handbook.pdf', 'file_path': 'handbook.pdf', 'file_type': 'application/pdf', 'file_size': 325352, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, hash='a0d6

In [110]:
response = agent.query("What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n What is the termination policy?")

Added user message to memory: What is the name of the company? 
 Who is the CEO of the company? 
 What is their vacation policy? 
 What is the termination policy?
=== Calling Function ===
Calling function: vector_tool with args: {"query": "name of the company"}
=== Function Output ===
Zania, Inc.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "CEO of the company"}
=== Function Output ===
Shruti Gupta is the CEO of Zania, Inc.
=== Calling Function ===
Calling function: vector_tool with args: {"query": "vacation policy"}
=== Function Output ===
The vacation policy includes the following key points:

1. Vacation granted during the first year of employment is prorated based on the hire date.
2. Eligible employees accrue vacation at a specified rate for every period worked, up to a maximum amount. Once this maximum is reached, no additional vacation will accrue until some of the accrued vacation is used.
3. Employees can begin using vacation immediately upon hir

In [115]:
pprint(response.get_formatted_sources())

('> Source (Doc id: 3742cc7d-3b65-4bed-bbdd-1108e8cb9c3f): Software programs '
 'purchased and provided by Zania, Inc. are to be used only for creating, '
 'researc...\n'
 '\n'
 '> Source (Doc id: 90f07bd5-87c3-49bc-ae90-f1692e8e06c6): The Company '
 'reserves the right to provide you with pay in lieu of notice in situations '
 'where job ...\n'
 '\n'
 '> Source (Doc id: 071a4c9f-8f5e-4ab9-bfa4-f66417a458e9): Closing Statement\n'
 'Thank you for reading our handbook. We hope it has provided you with an '
 'underst...\n'
 '\n'
 '> Source (Doc id: 02f356d0-671e-44ee-af6e-7eea4b5d1419): You are to use '
 'rental firms having existing relationships with the Company and, where '
 'feasible, h...\n'
 '\n'
 '> Source (Doc id: 2a148360-810f-4f73-945c-b3ad5f90e1be): Vacation granted '
 'during your first year of employment will be prorated based on your hire '
 'date.\n'
 '[...\n'
 '\n'
 '> Source (Doc id: efc244aa-c4f7-40e2-8ce5-008120f78029): [[The Company may '
 'elect to offer\n'
 'payment 

In [120]:
for node in response.source_nodes:
    print(node)

Node ID: 3742cc7d-3b65-4bed-bbdd-1108e8cb9c3f
Text: Software programs purchased and provided by Zania, Inc. are to
be used only for creating, researching, and processing materials for
Company use. By using Company hardware, software, and networking
systems you assume personal responsibility for their use and agree to
comply with this policy and other applicable Company policies, as well
as city, ...
Score:  0.380

Node ID: 90f07bd5-87c3-49bc-ae90-f1692e8e06c6
Text: The Company reserves the right to provide you with pay in lieu
of notice in situations where job or business needs warrant. Final Pay
The Company will pay separated employees in accordance with applicable
laws and other sections of this handbook.  Notify the Company if your
address changes during the calendar year in which resignation occurs
to ...
Score:  0.368

Node ID: 071a4c9f-8f5e-4ab9-bfa4-f66417a458e9
Text: Closing Statement Thank you for reading our handbook. We hope it
has provided you with an understanding of our m

In [146]:
task = agent.create_task("What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n What is the termination policy?")

In [147]:
len(agent.get_upcoming_steps(task.task_id)), len(agent.get_completed_steps(task.task_id)), len(agent.get_completed_tasks())

(1, 0, 0)

In [176]:
agent2 = StructuredPlannerAgent(
    agent_worker,
    tools=[doc_vector_tool], 
    verbose=True
)

In [177]:
agent2

In [178]:
plan_id = agent2.create_plan("What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n What is the termination policy?")

=== Initial plan ===
Extract company name:
What is the name of the company? -> [Company Name]
deps: []


Extract CEO name:
Who is the CEO of the company? -> [CEO Name]
deps: ['Extract company name']


Extract vacation policy:
What is their vacation policy? -> [Vacation Policy]
deps: ['Extract company name']


Extract termination policy:
What is the termination policy? -> [Termination Policy]
deps: ['Extract company name']




In [180]:
plan = agent2.state.plan_dict[plan_id]

for sub_task in plan.sub_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Extract company name =====
Expected output:  [Company Name]
Dependencies:  []
===== Sub Task Extract CEO name =====
Expected output:  [CEO Name]
Dependencies:  ['Extract company name']
===== Sub Task Extract vacation policy =====
Expected output:  [Vacation Policy]
Dependencies:  ['Extract company name']
===== Sub Task Extract termination policy =====
Expected output:  [Termination Policy]
Dependencies:  ['Extract company name']


In [181]:
next_tasks = agent2.state.get_next_sub_tasks(plan_id)
step_reponses = []

for sub_task in next_tasks:
    print(f"===== Sub Task {sub_task.name} =====")
    print("Expected output: ", sub_task.expected_output)
    print("Dependencies: ", sub_task.dependencies)

===== Sub Task Extract company name =====
Expected output:  [Company Name]
Dependencies:  []


In [ ]:
agent2.state.get_next_

1

In [ ]:
task = agent2.state.get_task()

TypeError: missing a required argument: 'task_id'

In [189]:
# create the react worker for reasoning
worker = FunctionCallingAgentWorker.from_tools(
    [doc_vector_tool], verbose=True
)

# wrap the worker in the top-level planner
agent = StructuredPlannerAgent(
    worker, tools=[doc_vector_tool], verbose=True
)

In [190]:
plan_id = agent.create_plan(
    "What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n What is the termination policy?"
)

=== Initial plan ===
Extract company name:
What is the name of the company? -> [Company Name]
deps: []


Extract CEO name:
Who is the CEO of the company? -> [CEO Name]
deps: ['Extract company name']


Extract vacation policy:
What is their vacation policy? -> [Vacation Policy]
deps: ['Extract company name']


Extract termination policy:
What is the termination policy? -> [Termination Policy]
deps: ['Extract company name']




In [192]:
import asyncio

while True:
    # are we done?
    next_tasks = agent.get_next_tasks(plan_id)
    if len(next_tasks) == 0:
        break

    # run concurrently for better performance
    step_responses = await asyncio.gather(
        *[agent.arun_task(task_id) for task_id in next_tasks]
    )
    for task_id in next_tasks:
        agent.mark_task_complete(plan_id, task_id)

    # refine the plan
    await agent.arefine_plan(
        "What is the name of the company? \n Who is the CEO of the company? \n What is their vacation policy? \n What is the termination policy?",
        plan_id,
    )

> Running step 321f474e-a250-4494-8488-f3370cd23915. Step input: What is the name of the company?
Added user message to memory: What is the name of the company?
=== Calling Function ===
Calling function: vector_tool with args: {"query": "company name"}
=== Function Output ===
Zania, Inc.
> Running step a0a40151-0800-41bb-9187-8351b8867a7d. Step input: None
=== LLM Response ===
The name of the company is Zania, Inc.
=== Refined plan ===
Extract CEO name:
Who is the CEO of the company Zania, Inc.? -> [CEO Name]
deps: []


Extract vacation policy:
What is the vacation policy of Zania, Inc.? -> [Vacation Policy]
deps: []


Extract termination policy:
What is the termination policy of Zania, Inc.? -> [Termination Policy]
deps: []


Compile company information:
Compile the company name, CEO name, vacation policy, and termination policy for Zania, Inc. -> Company Name: Zania, Inc., CEO: [CEO Name], Vacation Policy: [Vacation Policy], Termination Policy: [Termination Policy]
deps: ['Extract CE